In [1]:
# Source: https://github.com/prestinomills/aqueduct/blob/Know_Your_Community_Pipelines/civis/geohub/ActiveBusinessBlockgroupAggregation/Active_Business_Finalized_Script.py
# %load active_business_script.py
"""
Created on Wed May  1 08:51:03 2019
@author: myrfid041
"""
import geopandas as gpd
import os
import pandas as pd

import utils
from civis_aqueduct_utils.github import upload_file_to_github

from sodapy import Socrata
from arcgis.gis import GIS
from arcgis.features.summarize_data import join_features
from arcgis.features import FeatureLayer, FeatureLayerCollection
from copy import deepcopy
from IPython.display import display

import esri_credentials
lahub_user = esri_credentials.preston_user
lahub_pass = esri_credentials.preston_pw

#---Setting the Outputs
OUTPUT_FILE = "./Listing_of_Active_Businesses.csv"
output_layer_name = "a7236cc62ded454c94a64e9d80d6304a"

'''
ESRI stores the column names slightly differently (subject to 10 char limits)
Use dict to map and rename (key-value pair)
Key: dataframe's existing column name
Value: ESRI column name
'''
LAYER_RENAME_COLUMNS_DICT = {
    'Accommodation and Food Services': 'Accommodation_and_Food_Services',
    'Administrative and Support and Waste Management and Remediation Services': 'Administrative_and_Support_and_',
    'Agriculture, Forestry, Fishing and Hunting': 'Agriculture__Forestry__Fishing_',
    'Arts, Entertainment, and Recreation': 'Arts__Entertainment__and_Recrea',
    'Construction': 'Construction',
    'Educational Services': 'Educational_Services',
    'Finance and Insurance': 'Finance_and_Insurance',
    'Health Care and Social Assistance': 'Health_Care_and_Social_Assistan',
    'Information': 'Information',
    'Manufacturing': 'Manufacturing',
    'Medical Marijuana Collective': 'Medical_Marijuana_Collective',
    'Mining': 'Mining',
    'Not Classified': 'Not_Classified',
    'Other Services (except Public Administration)': 'Other_Services__except_Public_A',
    'Professional, Scientific, and Technical Services': 'Professional__Scientific__and_T',
    'Real Estate Rental and Leasing': 'Real_Estate_Rental_and_Leasing',
    'Retail Trade': 'Retail_Trade',
    'Transportation and Warehousing': 'Transportation_and_Warehousing',
    'Utilities': 'Utilities',
    'Wholesale Trade': 'Wholesale_Trade'                                               
}

In [2]:
new_updated_table = pd.read_parquet('./new_updated_table.parquet')

updated_values_dict = new_updated_table.set_index("GEOID10").to_dict(orient="index")

In [3]:
gis = GIS('https://lahub.maps.arcgis.com', username=lahub_user, password=lahub_pass)
flayer = gis.content.get(output_layer_name)
ActiveBusinesses_flayer = flayer.layers[0]
ActiveBusinesses_fset = ActiveBusinesses_flayer.query()
# Grab the features, save as a list
feature_list = ActiveBusinesses_fset.features

In [4]:
# Loop through each GEOID, then loop through the columns, and update the values
# Use try/except because there are some GEOIDs in the original sdf that aren't in our df (about 10ish)
# Leave these with original values, since we don't have updated values
updated_features = []
for i in range(0, len(feature_list)):
    original_feature = [f for f in feature_list][i]
    feature_to_be_updated = deepcopy(original_feature)
    attributes_to_update = feature_to_be_updated.attributes
    geoid = attributes_to_update["GEOID10"]
    try:
        for col in list(LAYER_RENAME_COLUMNS_DICT.values()):
            '''
            This doesn't work: 
            feature_to_be_updated[col] = updated_values_dict[geoid][col]

            feature_to_be_updated[col] is a features object...not a dict
            
            print(type(feature_list[0]))
            print(type(feature_list[0].attributes))
            
            Somehow, it won't let us key into the dict and change values
            But, since we earlier got into the attributes, we can update the values for each attribute
            https://gis.stackexchange.com/questions/394259/esri-notebook-select-and-append-records-to-table
            '''
            attributes_to_update[col] = int(updated_values_dict[geoid][col])
        updated_features.append(feature_to_be_updated)
        
    except:
        pass

In [5]:
utils.chunks(updated_features, 1000, ActiveBusinesses_flayer)

update successful
update successful
update successful
update successful
update successful
update successful
update successful
